# Importing libraries 

In [1]:
# %% Importing Libraries
import os
import sys
import pickle
import argparse
import time
import datetime
import random
from pathlib import Path

from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, balanced_accuracy_score

import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader, Dataset, RandomSampler, random_split, TensorDataset


from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

from pytorch_lightning.loggers import WandbLogger

import lightning as L
import lightning.pytorch as pl
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.tuner.tuning import Tuner
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.strategies import DeepSpeedStrategy
from lightning.pytorch.plugins.precision import DeepSpeedPrecisionPlugin

from deepspeed.ops.adam import DeepSpeedCPUAdam

import t5_encoder

# Custom library
sys.path.append('../process/')
from loadData import HTClassifierDataModule

sys.path.append('../architectures/')
from HTClassifier import HTClassifierModel

/root/miniconda3/envs/HT/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Creating directories if they don't exist
Path('../pickled/embeddings').mkdir(parents=True, exist_ok=True)

# Loading data

In [2]:
class Arguments():
    def __init__(self):
        self.model_name_or_path = 'johngiorgi/declutr-small'
        self.tokenizer_name_or_path = 'johngiorgi/declutr-small'
        self.data_dir = "../data/processed/TEXT/"
        self.demography = "merged"
        self.temp = 0.07 # Temperature for softmax
        self.max_seq_length = 512
        self.learning_rate = 3e-5 
        self.adam_epsilon = 1e-6
        self.warmup_steps = 0
        self.dropout = 0.3
        self.weight_decay = 0.01
        self.num_train_epochs = 1
        self.gradient_accumulation_steps = 4
        self.pad_to_max_length = True
        self.batch_size = 32
        self.output_dir = '../models/text-classifier-baselines/'
        self.overwrite = True
        self.local_rank = -1
        self.no_cuda = False

args = Arguments()

seed_everything(1111)

Global seed set to 1111


1111

In [3]:
dm = HTClassifierDataModule(args)
dm.setup()

../process/loadData.py:179: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_df = pd.read_csv(os.path.join(self.args.data_dir, self.args.demography + '.csv'), error_bad_lines=False, warn_bad_lines=False)
../process/loadData.py:179: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_df = pd.read_csv(os.path.join(self.args.data_dir, self.args.demography + '.csv'), error_bad_lines=False, warn_bad_lines=False)


In [4]:
args.num_classes = pd.read_csv(os.path.join(args.data_dir, args.demography + '.csv'), error_bad_lines=False, warn_bad_lines=False).VENDOR.nunique()

args.num_training_steps = len(dm.train_dataloader()) * 2
# Setting the warmup steps to 1/10th the size of training data
args.warmup_steps = int(len(dm.train_dataloader()) * 10/100)

/tmp/ipykernel_38874/4083842773.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  args.num_classes = pd.read_csv(os.path.join(args.data_dir, args.demography + '.csv'), error_bad_lines=False, warn_bad_lines=False).VENDOR.nunique()
/tmp/ipykernel_38874/4083842773.py:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  args.num_classes = pd.read_csv(os.path.join(args.data_dir, args.demography + '.csv'), error_bad_lines=False, warn_bad_lines=False).VENDOR.nunique()


# Loading the model

In [5]:
class HTClassifierModel(pl.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__()
        
        self.save_hyperparameters()
        if isinstance(args, tuple) and len(args) > 0: 
            self.args = args[0]
            self.hparams.learning_rate = self.args.learning_rate
            self.hparams.eps = self.args.adam_epsilon
            self.hparams.weight_decay = self.args.weight_decay
            self.hparams.model_name_or_path = self.args.model_name_or_path
            self.hparams.num_classes = self.args.num_classes
            self.hparams.num_training_steps = self.args.num_training_steps
            self.hparams.warmup_steps = self.args.warmup_steps
        
        # freeze
        self._frozen = False

        # Handling the padding token in distilgpt2 by substituting it with eos_token_id
        if self.hparams.model_name_or_path == "distilgpt2":
            config = AutoConfig.from_pretrained(self.hparams.model_name_or_path, num_labels=self.hparams.num_classes, output_attentions=True, output_hidden_states=True)
            self.model = AutoModelForSequenceClassification.from_pretrained(self.hparams.model_name_or_path, config=config)
            self.model.config.pad_token_id = self.model.config.eos_token_id
        else:
            config = AutoConfig.from_pretrained(self.hparams.model_name_or_path, num_labels=self.hparams.num_classes, output_attentions=True, output_hidden_states=True)
            self.model = AutoModelForSequenceClassification.from_pretrained(self.hparams.model_name_or_path, config=config)

    def forward(self, batch):
        # The batch contains the input_ids, the input_put_mask and the labels (for training)
        input_ids = batch[0]
        input_mask = batch[1]
        labels = batch[2]

        outputs = self.model(input_ids, attention_mask=input_mask, labels=labels)
        loss = outputs[0]
        logits = outputs[1]

        return outputs, loss, logits

    def training_step(self, batch, batch_nb):
        # the training step is a (virtual) method,specified in the interface, that the pl.LightningModule
        # class stipulates you to overwrite. This we do here, by virtue of this definition
        outputs = self(batch)  # self refers to the model, which in turn acceses the forward method
        train_loss = outputs[0]
        self.log_dict({"train_loss": train_loss, "learning_rate":self.hparams.learning_rate}, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return train_loss
        # the training_step method expects a dictionary, which should at least contain the loss

    def validation_step(self, batch, batch_nb):
        # the training step is a (virtual) method,specified in the interface, that the pl.LightningModule
        # class  wants you to overwrite, in case you want to do validation. This we do here, by virtue of this definition.

        outputs = self(batch)
        # self refers to the model, which in turn accesses the forward method

        # Apart from the validation loss, we also want to track validation accuracy  to get an idea, what the
        # model training has achieved "in real terms".
        val_loss = outputs[0]
        logits = outputs[1]
        labels = batch[2]

        # Evaluating the performance
        predictions = torch.argmax(logits, dim=1)
        balanced_accuracy = balanced_accuracy_score(labels.detach().cpu().numpy(), predictions.detach().cpu().numpy(), adjusted=True)
        macro_accuracy = f1_score(labels.detach().cpu().numpy(), predictions.detach().cpu().numpy(), average='macro')
        micro_accuracy = f1_score(labels.detach().cpu().numpy(), predictions.detach().cpu().numpy(), average='micro')
        weighted_accuracy = f1_score(labels.detach().cpu().numpy(), predictions.detach().cpu().numpy(), average='weighted')
        
        self.log_dict({"val_loss": val_loss, 'accuracy': balanced_accuracy, 'macro-F1': macro_accuracy, 'micro-F1': micro_accuracy, 'weighted-F1':weighted_accuracy}, 
                      on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return val_loss
    
    def test_step(self, batch, batch_nb):
        # the training step is a (virtual) method,specified in the interface, that the pl.LightningModule
        # class  wants you to overwrite, in case you want to do test. This we do here, by virtue of this definition.

        outputs = self(batch)
        # self refers to the model, which in turn accesses the forward method

        # Apart from the validation loss, we also want to track validation accuracy  to get an idea, what the
        # model training has achieved "in real terms".
        test_loss = outputs[0]
        logits = outputs[1]
        labels = batch[2]

        # Evaluating the performance
        predictions = torch.argmax(logits, dim=1)
        balanced_accuracy = balanced_accuracy_score(labels.detach().cpu().numpy(), predictions.detach().cpu().numpy(), adjusted=True)
        macro_accuracy = f1_score(labels.detach().cpu().numpy(), predictions.detach().cpu().numpy(), average='macro')
        micro_accuracy = f1_score(labels.detach().cpu().numpy(), predictions.detach().cpu().numpy(), average='micro')
        weighted_accuracy = f1_score(labels.detach().cpu().numpy(), predictions.detach().cpu().numpy(), average='weighted')
        
        self.log_dict({"test_loss": test_loss, 'accuracy': balanced_accuracy, 'macro-F1': macro_accuracy, 'micro-F1': micro_accuracy, 'weighted-F1':weighted_accuracy}, 
                      on_step=True, on_epoch=True, prog_bar=True, logger=True)
    
    def predict_step(self, batch, batch_nb):
        # the training step is a (virtual) method,specified in the interface, that the pl.LightningModule
        # class  wants you to overwrite, in case you want to do validation. This we do here, by virtue of this definition.

        outputs = self(batch)
        # self refers to the model, which in turn accesses the forward method

        # Apart from the validation loss, we also want to track validation accuracy  to get an idea, what the
        # model training has achieved "in real terms".
        val_loss = outputs[0]
        logits = outputs[1]
        labels = batch[2]

        predictions = torch.argmax(logits, dim=1)
        return predictions.detach().cpu().numpy()

    def configure_optimizers(self):
        # The configure_optimizers is a (virtual) method, specified in the interface, that the
        # pl.LightningModule class wants you to overwrite.

        # In this case we define that some parameters are optimized in a different way than others. In
        # particular we single out parameters that have 'bias', 'LayerNorm.weight' in their names. For those
        # we do not use an optimization technique called weight decay.

        no_decay = ['bias', 'LayerNorm.weight']

        optimizer_grouped_parameters = [{'params': [p for n, p in self.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay':self.hparams.weight_decay}, 
                                        {'params': [p for n, p in self.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
        # optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.eps)
        optimizer = DeepSpeedCPUAdam(optimizer_grouped_parameters, adamw_mode=True, lr=self.hparams.learning_rate, betas=(0.9, 0.999), eps=self.hparams.eps)

        # We also use a scheduler that is supplied by transformers.
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=self.hparams.num_training_steps)
        scheduler = {"scheduler": scheduler, "interval": "step", "frequency": 1}

        return [optimizer], [scheduler]

    def freeze(self) -> None:
        # freeze all layers, except the final classifier layers
        for name, param in self.model.named_parameters():
            if 'classifier' not in name:  # classifier layer
                param.requires_grad = False

        self._frozen = True

    def unfreeze(self) -> None:
        if self._frozen:
            for name, param in self.model.named_parameters():
                if 'classifier' not in name:  # classifier layer
                    param.requires_grad = True

        self._frozen = False

    def train_epoch_start(self):
        """pytorch lightning hook"""
        if self.current_epoch < self.hparams.nr_frozen_epochs:
            self.freeze()

        if self.current_epoch >= self.hparams.nr_frozen_epochs:
            self.unfreeze() 

In [6]:
model = HTClassifierModel(args).load_from_checkpoint("/workspace/persistent/human-trafficking/models/text-classifier-baselines/seed:1111/merged/declutr-small/final_model_new.pt").eval()

Some weights of the model checkpoint at johngiorgi/declutr-small were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at johngiorgi/declutr-small and are newly ini

# Splitting the dataset

In [7]:
df = pd.read_csv("../data/processed/TEXT/merged.csv")

text = df.TEXT.values.tolist()
vendors = df.VENDOR.values.tolist()

# Since the vendor IDs are not the current representations of the class labels, we remap these label IDs to avoid falling into out-of-bounds problem
vendors_dict = {}
i = 0
for vendor in vendors:
    if vendor not in vendors_dict.keys():
        vendors_dict[vendor] = i
        i += 1
        
train_df, test_df = train_test_split(df, test_size=0.20, random_state=1111)

# Extracting Embeddings

In [8]:
from transformers import AutoTokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name_or_path)

In [10]:
def extract_representations(test_df, demo, vendors_dict, pooling_type="mean", device="cpu", batch_size=32):
    # data_test = test_df[test_df.DEMO==demo]
    data_test = test_df
    data_test.replace({"VENDOR": vendors_dict}, inplace=True)

    text = data_test.TEXT.values.tolist()
    vendors = data_test.VENDOR.values.tolist()

    # Tokenizing the data with padding and truncation
    encodings = tokenizer(text, add_special_tokens=True, max_length=512, padding='max_length', return_token_type_ids=False, truncation=True, 
                               return_attention_mask=True, return_tensors='pt') 

    # Move the encodings to the device
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)
    labels = torch.tensor(vendors).to(device)

    # Combine the inputs into a TensorDataset.
    dataset = TensorDataset(input_ids, attention_mask, labels)
    test_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    pooled_output_list, labels_list = [], []
    
    pbar = tqdm(total=len(test_dataloader))
    with torch.no_grad():
        for batch in test_dataloader:
            attention_mask = batch[1]
            labels = batch[2]

            outputs = model(batch)

            # Extracting the output from last hidden state
            hidden_states = torch.stack(outputs[0][2])[-1]

            # Generating the pooled output
            if pooling_type == "mean":
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
                sum_embeddings = torch.sum(hidden_states * input_mask_expanded, 1)
                sum_mask = input_mask_expanded.sum(1)
                sum_mask = torch.clamp(sum_mask, min=1e-9)
                pooled_output = sum_embeddings / sum_mask
            elif pooling_type == "max":
                last_hidden_state = hidden_states
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
                last_hidden_state[input_mask_expanded == 0] = float("-inf")  # Set padding tokens to large negative value
                pooled_output = torch.max(last_hidden_state, 1)[0]
            else:
                # Mean-max pooling
                last_hidden_state = hidden_states
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
                sum_embeddings = torch.sum(hidden_states * input_mask_expanded, 1)
                sum_mask = input_mask_expanded.sum(1)
                sum_mask = torch.clamp(sum_mask, min=1e-9)
                mean_pooled_output = sum_embeddings / sum_mask
                last_hidden_state[input_mask_expanded == 0] = float("-inf")  # Set padding tokens to large negative value
                max_pooled_output = torch.max(last_hidden_state, 1)[0]
                pooled_output = torch.cat((mean_pooled_output, max_pooled_output), 1)

            pooled_output_list.append(pooled_output)
            labels_list.append(labels)
            pbar.update(1)
        pbar.close()

    # Concatenate the pooled outputs and labels into tensors
    pooled_outputs = torch.cat(pooled_output_list)
    labels = torch.cat(labels_list)

    return pooled_outputs, labels

In [11]:
for pooling in ["mean", "max", "mean-max"]:
    pooled_outputs, labels = extract_representations(train_df, "merged", vendors_dict, pooling_type=pooling)
    pooled_output_filename = "trained_traindata_declutr_" + pooling + ".pt"
    labels_filename = "trained_trainlabels_declutr_" + pooling + ".pt"

    torch.save(pooled_outputs, os.path.join(os.getcwd(), "../pickled/embeddings", pooled_output_filename))
    torch.save(labels, os.path.join(os.getcwd(), "../pickled/embeddings", labels_filename))

100%|██████████| 2190/2190 [3:05:21<00:00,  5.08s/it]  


In [12]:
for pooling in ["mean", "max", "mean-max"]:
    pooled_outputs, labels = extract_representations(test_df, "merged", vendors_dict, pooling_type=pooling)
    pooled_output_filename = "trained_testdata_declutr_" + pooling + ".pt"
    labels_filename = "trained_testlabels_declutr_" + pooling + ".pt"

    torch.save(pooled_outputs, os.path.join(os.getcwd(), "../pickled/embeddings", pooled_output_filename))
    torch.save(labels, os.path.join(os.getcwd(), "../pickled/embeddings", labels_filename))

100%|██████████| 548/548 [47:17<00:00,  5.18s/it]


# Loading the embeddings from an un-trained model

In [15]:
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer(args.model_name_or_path)

In [16]:
df = pd.read_csv("../data/processed/TEXT/merged.csv")

In [18]:
train_df, test_df = train_test_split(df, test_size=0.20, random_state=1111)

In [ ]:
embeddings = model.encode(train_df["TEXT"].to_list())

In [ ]:
labels = torch.tensor(train_df.VENDOR.to_list())
labels.shape, embeddings.shape

In [ ]:
torch.save(embeddings, os.path.join(os.getcwd(), "../pickled/embeddings", "untrained_styledata_train.pt"))
torch.save(labels, os.path.join(os.getcwd(), "../pickled/embeddings", "untrained_stylelabels_train.pt"))

In [ ]:
embeddings = model.encode(test_df["TEXT"].to_list())
labels = torch.tensor(test_df.VENDOR.to_list())
labels.shape, embeddings.shape

In [ ]:
torch.save(embeddings, os.path.join(os.getcwd(), "../pickled/embeddings", "untrained_styledata_test.pt"))
torch.save(labels, os.path.join(os.getcwd(), "../pickled/embeddings", "untrained_stylelabels_test.pt"))